# Byte Language Model

1. Copy this colab notebook into your google drive.
2. You need background knowledge for [Python](https://www.python.org/) and [NumPy](https://numpy.org/).
3. Run the cells yourself and tweak the code so that the byte-wise language model works as expected. Note that the current implementation does not perform any smoothing and thus leading to `inf` perplexity.
4. Save this colab notebook as a pdf via **Print** in the file menu and submit it to https://edu-portal.naist.jp/ under **NLP #3 & #4** of **2024 NAIST 4102 NLP** using the report submission portal. Please make sure that all the codes and the execution results are visible for the assessment.
5. Due date is **January 8th, 2025 JST**.

For help regarding [Colab](https://colab.research.google.com/) or any technical issues, ask our TA, Hongyu Sun <sun.hongyu.sg6@naist.ac.jp>.




In [1]:
#@markdown Please fill in your name, student id and email address.

name = 'Raturi Himanshu' #@param {type: 'string'}
stuent_id = '2411422' #@param {type: 'string'}
email = 'raturi.himanshu.rf4@naist.ac.jp' #@param {type: 'string'}

#@markdown ---

## Instructions

We will give 70 points for fixing a bug in the following code so that it can return perplexity values correctly, i.e., finite values, not, e.g., `Inf`. Additional points, i.e., maximum of 30, are given based on the ranking of the perpelxity, i.e., lower is better, among submissions.

* We will rank by the sum of three byte-wise perplexities, not word-wise perplexities, measured on three languages, Czech, German and Chinese.
* The scores are linearly computed among ranks. E.g., 30 is given to the submission with the lowest perplexity, 0 to those with the highest perplexity, and 15 to those in the middle of the ranks.
* Ties are grouped together as a bin, and their scores are computed by taking the average in the bins. For example, if a bin has 3 submissions with their linearly assigned scores of 13, 14 and 15, the average, i.e., $(13 + 14 + 15) / 3 = 14$, will be credited to those three.
* You can use any external libraries so long as you don't break APIs as documented/commented in the code. They are indicated by "DO NOT CHANGE" etc.
* If you tune any hyperparameters, please leave your experiments in this colab, e.g., your code and results. When tuning hyperparameters, do not tune them on the final test data, but use the development data.

### Penalties

The byte-wise perplexity will be penalyzed by adding a penaty term when the cumulative product of sum of probabilities is not closer to 1 using the following formula:

$penalty = \max(\operatorname{abs}(1 - \prod_{t=1}^{T} \sum_{y=0}^{255} p(y | x_{<t})) - 0.001, 0) \times 100$

where $x_t$ is a byte in a file $\boldsymbol{x}$ at position $t$. Note that your language model must be probabilitistic in that sum over all byte values must be equal to one given any histories.

### Extras

Those who tried "unique methods" will be given at most 10 points in addition to the base and ranking points. The uniqueness is determined whether a submission employs a smoothing method other students have not tried. However, maximum 10 points will be deducted when violating rules, e.g., changing part of the codes/APIs which should not be modified, or tuning hyperparameters on the test data.

## Download datasets

We will down load the datasets to train and test your language models. The dataset is extracted from [WMT 2024 Shared Task](https://www2.statmt.org/wmt24/translation-task.html).


In [2]:
# Download the file to "/content" directory.
!gdown 15P-8w5Y90qG3GOoyKzq4PrKUjN6LDFpV
!gdown 1wVaCzs2fOT7ZWfwrFj_duI0yKU_4Qcqq
!gdown 1C-d9-J_BdFd7ADoqre5YYkYkdq0p-Dfh

# Unzip it.
!unzip -o byte-language-model-2024.de.zip
!unzip -o byte-language-model-2024.hi.zip
!unzip -o byte-language-model-2024.ru.zip

# Create smaller training datasets comprising 3000 sentences for development
# purposes.
! head -3000 byte-language-model-2024.de/news-commentary-v18.de.txt > byte-language-model-2024.de/news-commentary-v18-small.de.txt
! head -3000 byte-language-model-2024.hi/news-commentary-v18.hi.txt > byte-language-model-2024.hi/news-commentary-v18-small.hi.txt
! head -3000 byte-language-model-2024.ru/news-commentary-v18.ru.txt > byte-language-model-2024.ru/news-commentary-v18-small.ru.txt

Downloading...
From (original): https://drive.google.com/uc?id=15P-8w5Y90qG3GOoyKzq4PrKUjN6LDFpV
From (redirected): https://drive.google.com/uc?id=15P-8w5Y90qG3GOoyKzq4PrKUjN6LDFpV&confirm=t&uuid=94f2d80e-89e9-4ba4-893c-3fa546b38b0e
To: /content/byte-language-model-2024.de.zip
100% 27.0M/27.0M [00:00<00:00, 30.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wVaCzs2fOT7ZWfwrFj_duI0yKU_4Qcqq
To: /content/byte-language-model-2024.hi.zip
100% 673k/673k [00:00<00:00, 36.7MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1C-d9-J_BdFd7ADoqre5YYkYkdq0p-Dfh
From (redirected): https://drive.google.com/uc?id=1C-d9-J_BdFd7ADoqre5YYkYkdq0p-Dfh&confirm=t&uuid=7037d76f-fa55-48ab-87f6-2c2fe904f178
To: /content/byte-language-model-2024.ru.zip
100% 29.8M/29.8M [00:00<00:00, 58.4MB/s]
Archive:  byte-language-model-2024.de.zip
  inflating: byte-language-model-2024.de/news-commentary-v18.de.txt  
  inflating: byte-language-model-2024.de/wmttest2023.de.txt  
  inflating: byte-

## Verifies the extracted files

We will compute the md5 checksums to make sure that you are using the correct files. You will observe the following outputs:

```bash
c5cf2b0f973eccb12e45a56039d44f99  byte-language-model-2024.de/news-commentary-v18.de.txt
383597f984448acb4d4d09c839055d26  byte-language-model-2024.de/news-commentary-v18-small.de.txt
b58c3944859f9df96271e1f56d5a97e7  byte-language-model-2024.de/wmttest2023.de.txt
9efc37e56a73014c2e17856a0d09b4bd  byte-language-model-2024.de/wmttest2024.de.txt
acd6b1f3391ae49bb7ab7df0eae2ac78  byte-language-model-2024.hi/news-commentary-v18.hi.txt
6971694001b2ae62e01c8502cd61bba1  byte-language-model-2024.hi/news-commentary-v18-small.hi.txt
f410731105f7cd313148069f096e34c3  byte-language-model-2024.hi/wmttest2023.hi.txt
70c511e65e6e10de9daccc5e0a2b103f  byte-language-model-2024.hi/wmttest2024.hi.txt
2516f578eea6428dd3e6d9a0c2135bf8  byte-language-model-2024.ru/news-commentary-v18.ru.txt
f976485f7baaf7a66159f23ab3c69025  byte-language-model-2024.ru/news-commentary-v18-small.ru.txt
7620468544b08798fb5172e6406826c7  byte-language-model-2024.ru/wmttest2023.ru.txt
ec00fc9496a3e1077baf328c82dc2421  byte-language-model-2024.ru/wmttest2024.ru.txt
```

Note that we will use `news-commentary-v18.{de,hi,ru}.txt` for training and `wmttest2024.{de,hi,ru}.txt` for the final testing. `wmttest2023.{de,hi,ru}.txt` will be used for your development purposes, e.g., debugging or fine tuning hyperparameters.

In [3]:
# Runs md5sum for the unzipped file.
# please make sure tat the hash codes are the same.

!md5sum byte-language-model-2024.*/*

c5cf2b0f973eccb12e45a56039d44f99  byte-language-model-2024.de/news-commentary-v18.de.txt
383597f984448acb4d4d09c839055d26  byte-language-model-2024.de/news-commentary-v18-small.de.txt
b58c3944859f9df96271e1f56d5a97e7  byte-language-model-2024.de/wmttest2023.de.txt
9efc37e56a73014c2e17856a0d09b4bd  byte-language-model-2024.de/wmttest2024.de.txt
acd6b1f3391ae49bb7ab7df0eae2ac78  byte-language-model-2024.hi/news-commentary-v18.hi.txt
6971694001b2ae62e01c8502cd61bba1  byte-language-model-2024.hi/news-commentary-v18-small.hi.txt
f410731105f7cd313148069f096e34c3  byte-language-model-2024.hi/wmttest2023.hi.txt
70c511e65e6e10de9daccc5e0a2b103f  byte-language-model-2024.hi/wmttest2024.hi.txt
2516f578eea6428dd3e6d9a0c2135bf8  byte-language-model-2024.ru/news-commentary-v18.ru.txt
f976485f7baaf7a66159f23ab3c69025  byte-language-model-2024.ru/news-commentary-v18-small.ru.txt
7620468544b08798fb5172e6406826c7  byte-language-model-2024.ru/wmttest2023.ru.txt
ec00fc9496a3e1077baf328c82dc2421  byte-lang

## Import libraries

Adds necessary imports here if you want to use additional libraries.

In [4]:
# Import libraries used in this colab.
import collections
from typing import Any, Dict, List, Tuple

from google.colab import files
import numpy as np

## Language model implementation

`ByteLM` is a language model class which loads a training data, estimate parameters and test it on a file to compute byet-wise perplexity.

In [5]:
class ByteLM:
  """Byte language model.

  This is a very naive language model, in which byte-wise ngram probabilities
  are estimated by maximum-likelihood without considering an issue of
  out-of-vocabulary.

  You may want to tweak `__init__`, `initial_state` and `logprob` methods to
  alleviate the problem. However, do not change `perplexity` for a fair
  comparison with other models implemented by other students. When changing part
  of the codes, please try to make it readable by using appropriate variable
  names or adding comments. Feel free to add additional methods, if necessary.

  Usages:
    ```python
    lm = ByteLM(path/to/train/data)
    perplexity, prob = lm.perplexity(path/to/test/data)
    ```
  """

  # DO NOT CHANGE BOS VALUE.
  # 0 will never appear in a text, thus, used it as a special symbol for
  # a beginning-of-sentence symbol, i.e., BOS.
  BOS: int = 0

  def __init__(self, filename: str, order: int=3, smoothing: float=0.1) -> None:
    """Initializes `ByteLM`.

    You can change the arguments for this method if necessary, e.g., adding
    hyperparameters to this model.

    Args:
      filename: str, text file to train this language model.
      order: int, the n-gram order that should be greater than 1.
    """
    if order <= 1:
      raise ValueError(f'`order` must be greater than 1: {order}')
    self.order = order
    self.smoothing = smoothing
    # You can try revise the code in this method to fix a bug and to achieve
    # lower perplexity.

    # Collect n-gram counts. The dictionary comprises a key of tuple of
    # integers, i.e., (n-1)-gram, and its associated value of 256-dimensonal
    # vector, i.e., counts for the following chars.
    ngram_counts = collections.defaultdict(lambda: np.zeros([256]))
    with open(filename, 'br') as f:
      for line in f:  # read as a byte string.
        buffer = [self.BOS] + list(line)  # `buffer` is now a list of integers.
        for n in range(1, self.order + 1):
          for i in range(len(buffer) - n + 1):
            ngram = buffer[i:i + n]
            ngram_counts[tuple(ngram[:-1])][ngram[-1]] += 1

    # Maximum likelihood estimate for language model.
    self.ngrams: Dict[Tuple[int], np.ndarray] = {}
    for context, counts in ngram_counts.items():
      smoothed_counts = counts + self.smoothing
      probs = smoothed_counts / np.sum(smoothed_counts)
      # Computes log probabilities, but assigns -inf for zero probabilities.
      log_probs = np.log(probs)
      self.ngrams[context] = log_probs

  def initial_state(self) -> Any:
    """Returns an initial state for language model computation.

    You can change the code in this method, but keep the API, e.g, input
    arguments, so that `perplexity()` method works as expected.

    Returns:
      A state representation for log probabilities computation.
    """
    # You can revise the code here for lower perplexity.
    return [self.BOS]

  def logprob(self, state: Any, x: int) -> Tuple[np.ndarray, Any]:
    """Returns log probabilities for the current input byte.

    You can change the code in this method, but keep the API, e.g, input
    arguments, so that `perplexity()` method works as expected.

    Args:
      state: A state to compute log probability.
      x: int, the current byte to compute `p(y | state, x)`.
    Returns:
      A pair of (log_probs, next_state) where `log_probs` is `np.ndarray` of log
      probabilities p(y | state, x) of all bytes y, and `next_state` is a new
      state for the next log probability computation with a new input. Note that
      `log_probs[y]` is equal to `log p(y | state, x)`,
      `log_probs.shape == (256,)`, `np.exp(log_probs) >= 0` and
      `np.sum(np.exp(log_probs)) == 1`.
    """
    # You many want to revise the code in this method to achieve lower
    # perplexity.

    # Backoff to lower order when necessary.
    state = (state + [x])[-self.order + 1:]
    for i in range(len(state), 0, -1):
       context = state[-i:]
       assert len(context) < self.order
       ret = self.ngrams.get(tuple(context), None)
       if ret is not None:
         return ret, context

    # Backoff to unigram.
    ret = self.ngrams.get((), None)
    assert ret is not None
    return ret, []

  def perplexity(self, filename: str) -> Tuple[float, float]:
    """Computes perplexity for text data.

    DO NOT CHANGE THE API OR CODE IN THIS METHOD.

    Args:
      filename: str, text file to compute perplexity.
    Returns:
      A pair (perplexity, prob) where `perplexity` is the perplexity computed
      for `filename`. `prob` is the cumulative product of probabilities of all
      the bytes in `filename` to verify that this language model is
      probabilistic or not. `prob` should be close to 1, otherwise, this is not
      a language model.
    """
    # Cumulative log_prob for perplexity computation.
    cumulative_log_prob = 0.0
    # Verify the distribution so that this language model is probabilistic.
    prob = 1.0
    # Total number of bytes.
    total_bytes = 0
    with open(filename, 'br') as f:
      for line in f:
        state = self.initial_state()
        prev_x = self.BOS
        for x in line:
          log_probs, state = self.logprob(state, prev_x)
          assert log_probs.size == 256, f"expected 256, got: {log_probs.size}"
          cumulative_log_prob += log_probs[x]

          probs = np.exp(log_probs)
          assert (probs >= 0).all(), "expected greater than or equal to zero."
          prob *= np.sum(probs)  # Sum of `probs` should be close to 1.

          prev_x = x

        total_bytes += len(line)

    return np.exp(-cumulative_log_prob / total_bytes), prob

## Run and report perplexity

Please run the following code block to report the final perplexity of three language models. You can change the hyperparameters to the language model, e.g., additional arguments for constructing three models, `model_de`, `model_hi` and `model_ru`. However, do not change the rest of the code for a fair comparison with others.

We will rank your "adjusted" perplexity results from three language models after adding penalty terms.

In [6]:
# Computes md5 hash to make sure that the correct datasets are used for training
# and testing.
!md5sum byte-language-model-2024.*/*

# Train a Czech language model using German data. Note that you can modify the
# arguments to `ByteLM` to set hyperparameters to the language model.
model_de = ByteLM("byte-language-model-2024.de/news-commentary-v18.de.txt")

# Train a Hindi language model using German data. Note that you can modify the
# arguments to `ByteLM` to set hyperparameters to the language model.
model_hi = ByteLM("byte-language-model-2024.hi/news-commentary-v18.hi.txt")

# Train a Russian language model using Chinese data. Note that you can modify
# the arguments to `ByteLM` to set hyperparameters to the language model.
model_ru = ByteLM("byte-language-model-2024.ru/news-commentary-v18.ru.txt")

# DO NOT CHANGE THE FOLLOWING CODES FOR FAIR COMPARISON.

# Testing on Czech data using the language model trained by German data.
perp_de, prob_de = model_de.perplexity("byte-language-model-2024.de/wmttest2024.de.txt")

# Testing on German data using the language model trained by Hindi data.
perp_hi, prob_hi = model_hi.perplexity("byte-language-model-2024.hi/wmttest2024.hi.txt")

# Testing on Chinese data using the language model trained by Russain data.
perp_ru, prob_ru = model_ru.perplexity("byte-language-model-2024.ru/wmttest2024.ru.txt")

# Computes total perplexity from three languages.
perp = perp_de + perp_hi + perp_ru

# Computes penalties and simply sums them.
penalty_de = np.maximum(np.abs(1 - prob_de) - 0.001, 0) * 100
penalty_hi = np.maximum(np.abs(1 - prob_hi) - 0.001, 0) * 100
penalty_ru = np.maximum(np.abs(1 - prob_ru) - 0.001, 0) * 100
adjusted_perp = perp + penalty_de + penalty_hi + penalty_ru

# Print out computation results. "adjusted perplexity" is used for ranking.
print(f"de perplexity: {perp_de} prob: {prob_de} penalty: {penalty_de}")
print(f"hi perplexity: {perp_hi} prob: {prob_hi} penalty: {penalty_hi}")
print(f"ru perplexity: {perp_ru} prob: {prob_ru} penalty: {penalty_ru}")
print(f"total perplexity: {perp}")
print(f"adjusted: {adjusted_perp:.4f}")

c5cf2b0f973eccb12e45a56039d44f99  byte-language-model-2024.de/news-commentary-v18.de.txt
383597f984448acb4d4d09c839055d26  byte-language-model-2024.de/news-commentary-v18-small.de.txt
b58c3944859f9df96271e1f56d5a97e7  byte-language-model-2024.de/wmttest2023.de.txt
9efc37e56a73014c2e17856a0d09b4bd  byte-language-model-2024.de/wmttest2024.de.txt
acd6b1f3391ae49bb7ab7df0eae2ac78  byte-language-model-2024.hi/news-commentary-v18.hi.txt
6971694001b2ae62e01c8502cd61bba1  byte-language-model-2024.hi/news-commentary-v18-small.hi.txt
f410731105f7cd313148069f096e34c3  byte-language-model-2024.hi/wmttest2023.hi.txt
70c511e65e6e10de9daccc5e0a2b103f  byte-language-model-2024.hi/wmttest2024.hi.txt
2516f578eea6428dd3e6d9a0c2135bf8  byte-language-model-2024.ru/news-commentary-v18.ru.txt
f976485f7baaf7a66159f23ab3c69025  byte-language-model-2024.ru/news-commentary-v18-small.ru.txt
7620468544b08798fb5172e6406826c7  byte-language-model-2024.ru/wmttest2023.ru.txt
ec00fc9496a3e1077baf328c82dc2421  byte-lang

## Add your codes if necessary

You can add arbitrary codes here, e.g., running experiments on smaller training data, i.e., `byte-language-model-2024.de/news-commentary-v18-small.de.txt` and/or `byte-language-model-2024.hi/news-commentary-v18-small.hi.txt`, together with development data, `byte-language-model-2024.de/wmttest2023.de.txt` and/or `byte-language-model-2024.hi/wmttest2023.hi.txt`. You can easily add your code by clicking `+ Code` at the top of this notebook, near the menu bar.

When you tweak any hyperparameters of your model, you may keep some code run results as a justification of the choices, e.g., run results on the development datasets. However, do not tune any hyperparameters on the test data.

In [7]:
# You can add your code here for your testing purposes, e.g., runs on
# development data to tweak your codes in `ByteLM` or find hyperparameters.
# However, do not tune on the test data.

In [8]:
class ByteLM:
  BOS: int = 0

  def __init__(self, filename: str, order: int=3, smoothing: float=1.0) -> None:
    if order <= 1:
      raise ValueError(f'`order` must be greater than 1: {order}')
    self.order = order
    self.smoothing = smoothing

    ngram_counts = collections.defaultdict(lambda: np.zeros([256]))
    with open(filename, 'br') as f:
      for line in f:  # read as a byte string.
        buffer = [self.BOS] + list(line)  # `buffer` is now a list of integers.
        for n in range(1, self.order + 1):
          for i in range(len(buffer) - n + 1):
            ngram = buffer[i:i + n]
            ngram_counts[tuple(ngram[:-1])][ngram[-1]] += 1

    self.ngrams: Dict[Tuple[int], np.ndarray] = {}
    for context, counts in ngram_counts.items():
      smoothed_counts = counts + self.smoothing
      probs = smoothed_counts / np.sum(smoothed_counts)
      log_probs = np.log(probs)
      self.ngrams[context] = log_probs

  def initial_state(self) -> Any:
    return []

  def logprob(self, state: Any, x: int) -> Tuple[np.ndarray, Any]:
    state = (state + [x])[-self.order + 1:]
    for i in range(len(state), 0, -1):
       context = state[-i:]
       assert len(context) < self.order
       ret = self.ngrams.get(tuple(context), None)
       if ret is not None:
         return ret, context

    # Backoff to unigram.
    ret = self.ngrams.get((), None)
    assert ret is not None
    return ret, []

  def perplexity(self, filename: str) -> Tuple[float, float]:
    cumulative_log_prob = 0.0
    prob = 1.0
    total_bytes = 0
    with open(filename, 'br') as f:
      for line in f:
        state = self.initial_state()
        prev_x = self.BOS
        for x in line:
          log_probs, state = self.logprob(state, prev_x)
          assert log_probs.size == 256, f"expected 256, got: {log_probs.size}"
          cumulative_log_prob += log_probs[x]

          probs = np.exp(log_probs)
          assert (probs >= 0).all(), "expected greater than or equal to zero."
          prob *= np.sum(probs)  # Sum of `probs` should be close to 1.

          prev_x = x

        total_bytes += len(line)

    return np.exp(-cumulative_log_prob / total_bytes), prob

In [9]:
# Define a range of smoothing values to test
smoothing_values = [0.1, 0.5, 1.0, 2.0, 5.0]

# Store perplexity results for each smoothing value
results = []

for smoothing in smoothing_values:
  print(f"Testing with smoothing value: {smoothing}")
  model_de = ByteLM("byte-language-model-2024.de/news-commentary-v18-small.de.txt", smoothing=smoothing)
  perp_de, prob_de = model_de.perplexity("byte-language-model-2024.de/wmttest2023.de.txt")
  results.append((smoothing, perp_de))
  print(f"Perplexity: {perp_de}, Probability: {prob_de}")

# Find the best smoothing value
best_smoothing = min(results, key=lambda x: x[1])
print(f"Best smoothing value: {best_smoothing[0]} with perplexity: {best_smoothing[1]}")

Testing with smoothing value: 0.1
Perplexity: 9.939325568502968, Probability: 0.9999999999999979
Testing with smoothing value: 0.5
Perplexity: 10.768516546223317, Probability: 0.9999999999990565
Testing with smoothing value: 1.0
Perplexity: 11.683562050535413, Probability: 0.9999999999999977
Testing with smoothing value: 2.0
Perplexity: 13.229887698336501, Probability: 0.9999999999960931
Testing with smoothing value: 5.0
Perplexity: 16.921358933195567, Probability: 0.9999999999977742
Best smoothing value: 0.1 with perplexity: 9.939325568502968
